In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import os
import seaborn as sns
import scipy

In [ ]:
sales_data = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
items = pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
item_cat = pd.read_csv("../input/competitive-data-science-predict-future-sales/item_categories.csv")
shops = pd.read_csv("../input/competitive-data-science-predict-future-sales/shops.csv")
sample_submission = pd.read_csv("../input/competitive-data-science-predict-future-sales/sample_submission.csv")
test_data = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
sales_data.head(3)

In [ ]:
test_data.head(3)

In [ ]:
sample_submission.head(3)

In [ ]:
def basic_eda(df):
    print("----------TOP 5 RECORDS--------")
    print(df.head(5))
    print("----------INFO-----------------")
    print(df.info())
    print("----------Describe-------------")
    print(df.describe())
    print("----------Columns--------------")
    print(df.columns)
    print("----------Data Types-----------")
    print(df.dtypes)
    print("-------Missing Values----------")
    print(df.isnull().sum())
    print("-------NULL values-------------")
    print(df.isna().sum())
    print("-----Shape Of Data-------------")
    print(df.shape)

In [ ]:
print("=============================Sales Data=============================")
basic_eda(sales_data)
print("=============================Test data=============================")
basic_eda(test_data)
print("=============================Item Categories=============================")
basic_eda(item_cat)
print("=============================Items=============================")
basic_eda(items)
print("=============================Shops=============================")
basic_eda(shops)
print("=============================Sample Submission=============================")
basic_eda(sample_submission)

In [ ]:
sales_data['date'] = pd.to_datetime(sales_data['date'],format = '%d.%m.%Y')

In [ ]:
sales_data.head(3)

In [ ]:
dataset = sales_data.pivot_table(index = ['shop_id','item_id'],values = ['item_cnt_day'],columns = ['date_block_num'],fill_value = 0,aggfunc='sum')

In [ ]:
dataset.head(3)

In [ ]:
dataset.reset_index(inplace = True)

In [ ]:
dataset.head(3)

In [ ]:
dataset = pd.merge(test_data,dataset,on = ['item_id','shop_id'],how = 'left')

In [ ]:
dataset.head()

In [ ]:
dataset.fillna(0,inplace = True)
dataset.head()

In [ ]:
dataset.drop(['shop_id','item_id','ID'],inplace = True, axis = 1)
dataset.head()

In [ ]:
X_train = np.expand_dims(dataset.values[:,:-1],axis = 2)
y_train = dataset.values[:,-1:]

X_test = np.expand_dims(dataset.values[:,1:],axis = 2)

print(X_train.shape,y_train.shape,X_test.shape)

In [ ]:

from keras import optimizers
from keras.models import Sequential, Model
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [ ]:
model_lstm = Sequential()
model_lstm.add(LSTM(units = 64,input_shape = (X_train.shape[1], X_train.shape[2])))
model_lstm.add(Dropout(0.4))
model_lstm.add(Dense(1))

model_lstm.compile(loss = 'mse', optimizer = 'adam', metrics = ['mean_squared_error'])
model_lstm.summary()

In [ ]:
history_lstm = model_lstm.fit(X_train,y_train,batch_size = 4096,epochs = 10)

In [ ]:
plt.plot(history_lstm.history['loss'], color ='b', label="Training loss")
plt.legend(loc='best',shadow=True)

In [ ]:
submission_pfs = model_lstm.predict(X_test)
submission_pfs = submission_pfs.clip(0,20)
submission = pd.DataFrame({'ID':test_data['ID'], 'item_cnt_month':submission_pfs.ravel()})
submission.to_csv('sub_pfs.csv', index=False)

In [ ]:
submission.head(3)

In [ ]:
submission.shape, test_data.shape